In [ ]:
%matplotlib inline


Basic neuroquery example: downloading a trained model and making a few queries
==============================================================================

This example shows basic neuroquery functionality:

- download a trained neuroquery model
- query it with some text
- have a look at the result: a brain map and a list of similar or related
    terms.

The model used here is the same as the one deployed on the neuroquery website
( https://neuroquery.saclay.inria.fr ), and this example shows how to easily
reproduce the website's functionality in a Python script.


Download the vocabulary and learned coefficients of a NeuroQuery model
----------------------------------------------------------------------



In [ ]:
from neuroquery import datasets

neuroquery_data = datasets.fetch_neuroquery_model()
print(neuroquery_data)

Load the NeuroQuery model
-------------------------



In [ ]:
from neuroquery import text_to_brain

encoder = text_to_brain.TextToBrain.from_data_dir(neuroquery_data)

Query the model and display the results
---------------------------------------



In [ ]:
# taken from Wikipedia
query = "Aphasia is an inability to comprehend or formulate language"

response = encoder(query)
print(response.keys())

The "z_map" entry of the results is a brain map showing the anatomical
regions that are most strongly associated with the query in the neuroimaging
literature. It is a `Nifti1Image` which can be saved, displayed, etc.



In [ ]:
from nilearn import plotting

print(type(response["z_map"]))
plotting.plot_stat_map(
    response["z_map"], display_mode="z", title="aphasia", threshold=3.)

In [ ]:
# Display the map on the cortical surface:
# (in a Jupyter notebook,
# `.open_in_browser()` can be ommitted to open an inline view)
plotting.view_img_on_surf(response["z_map"], threshold=3.).open_in_browser()

In [ ]:
# Or open interactive viewer:
# (in a Jupyter notebook,
# `.open_in_browser()` can be ommitted to open an inline view)
plotting.view_img(response["z_map"], threshold=3.).open_in_browser()

"similar_words" is a DataFrame containing terms that are related to the
query. For each related terms, three numbers are provided.
"similarity" is the strength of the association between the term and the
query, according to co-occurrence statistics in the literature.



In [ ]:
print("Most similar terms:\n")
print(response["similar_words"].sort_values(
    by="similarity", ascending=False).head())

"weight_in_brain_map" is the importance of the term in the brain map. It
depends both on the similarity with the query and on the strength of the link
between this term and brain activity. Terms that are similar to the query and
have a strong association with brain activity in the literature get a higher
weight.



In [ ]:
print("\nMost important terms for brain map prediction:\n")
print(response["similar_words"].sort_values(
    by="weight_in_brain_map", ascending=False).head())

"weight_in_query" is the importance of the term in the query itself. It
reflects the number of times each term appears in the query (reweighted so
that very common, uninformative words get a lower weight). For example, terms
that do not appear in the query get a "weight_in_query" of 0.



In [ ]:
print("\nTerms recognized in the query:\n")
print(response["similar_words"].query("weight_in_query != 0").sort_values(
    by="weight_in_query", ascending=False).head())

Finally, "highlighted_text" contains the text of the query itself, with
markups that indicates which terms were recognized and used by the model
(usually terms related to neuroimaging):



In [ ]:
print(response["highlighted_text"])

# `print_highlighted_text` can help display it in a terminal for debugging.

from neuroquery.tokenization import print_highlighted_text

print("\n")
print_highlighted_text(response["highlighted_text"])

In [ ]:
plotting.show()